In [1]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanSquaredPercentageError
from sktime.split import ExpandingWindowSplitter

# from sktime.datasets import load_longley
import pandas as pd
from src.data.data_loader import load_data
from src.data.data_cleaner import clean_data

In [13]:
def load_diabetes_data():
    df = clean_data(load_data())
    patient_data = df[df["p_num"] == "p01"].copy()
    patient_data = patient_data.fillna(0)

    # Set datetime index
    patient_data["time"] = pd.to_datetime(patient_data["time"])
    # patient_data = patient_data.set_index('time')

    y = patient_data["bg-0:00"]
    X = patient_data[["insulin-0:00", "carbs-0:00", "hr-0:00"]]

    return y, X


# y, X = load_longley()

# Update your dataset loaders
dataset_loaders = [load_diabetes_data]

In [14]:
benchmark = ForecastingBenchmark()
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=1),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=1),
    estimator_id="NaiveForecaster-last-v1",
)

In [15]:
steps_per_hour = 12
hours_to_forecast = 6

cv_splitter = ExpandingWindowSplitter(
    initial_window=steps_per_hour,
    step_length=steps_per_hour,
    fh=steps_per_hour * hours_to_forecast,
)

In [16]:
scorers = [MeanSquaredPercentageError()]

In [17]:
for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

In [18]:
results_df = benchmark.run("./forecasting_results.csv")
results_df.T

/var/folders/lv/s2fdnq713td8c15d7dlzjk6c0000gn/T/ipykernel_55684/1814209915.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  patient_data['time'] = pd.to_datetime(patient_data['time'])
/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/venv/lib/python3.9/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/var/folders/lv/s2fdnq713td8c15d7dlzjk6c0000gn/T/ipykernel_55684/1814209915.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  patient_data['time'] = pd.to_datetime(patient_data['time'])


,0,1
validation_id,[dataset=load_diabetes_data]_[cv_splitter=Expa...,[dataset=load_diabetes_data]_[cv_splitter=Expa...
model_id,NaiveForecaster-last-v1,NaiveForecaster-mean-v1
runtime_secs,23.361416,19.860873
MeanSquaredPercentageError_fold_0_test,0.061471,0.018221
MeanSquaredPercentageError_fold_1_test,0.225385,0.075764
...,...,...
MeanSquaredPercentageError_fold_695_test,0.487185,0.205711
MeanSquaredPercentageError_fold_696_test,0.151235,0.016905
MeanSquaredPercentageError_fold_697_test,0.865333,0.7946
MeanSquaredPercentageError_mean,128702640355733392367318468657152.0,99752492011008571189352244183040.0
